In [1]:
import sys
sys.path.append(sys.path[0][:sys.path[0].find('DVML-P7') + len('DVML-P7')])
import re
import numpy as np
import pandas as pd
from Code.UtilityFunctions.run_query import run_query
from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.wikidata_functions import get_name_of_location_with_long_lat, wikidata_query, categories_dict_singular
from collections import Counter

In [90]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)

coordinates = (biz["longitude"].apply(round, args=(2,)).astype(str) + "," + biz["latitude"].apply(round, args=(2,)).astype(str)).unique()
coordinates

array(['-119.71,34.43', '-90.34,38.55', '-110.88,32.22', ...,
       '-90.05,30.02', '-90.23,38.66', '-75.41,40.2'], dtype=object)

In [91]:
coordinates[0:10]

array(['-119.71,34.43', '-90.34,38.55', '-110.88,32.22', '-75.16,39.96',
       '-75.47,40.34', '-87.06,36.27', '-90.34,38.63', '-82.73,27.77',
       '-90.32,38.57', '-86.77,36.21'], dtype=object)

In [92]:
coordinates[0:1][0]

'-119.71,34.43'

In [93]:
coordinates_df = pd.DataFrame(coordinates, columns=['coordinates']).head(40)

In [10]:
for i in coordinates_df.itertuples():
    if get_name_of_location_with_long_lat(i.coordinates) is None:
        print(i.coordinates, get_name_of_location_with_long_lat(i.coordinates))

In [94]:
df = pd.DataFrame(coordinates_df['coordinates'].apply(lambda x: [x] + list(get_name_of_location_with_long_lat(x))).tolist(), columns=['coordinates','mapped_qid', 'qid_label'])
df.to_csv(path_or_buf='/home/ubuntu/DVML-P7/Development/location_mappings.csv',index=False)

In [17]:
print(f"Number of unique qid's: {len(location_mappings.mapped_qid.unique())}\nNumber of unique city names: {len(location_mappings.qid_label.unique())}")

Number of unique qid's: 693
Number of unique city names: 684


In [58]:
location_mappings_df = pd.DataFrame(coordinates_df['coordinates'].apply(lambda x: [x] + list(get_name_of_location_with_long_lat(x))).tolist(), columns=['coordinates','city', 'cityLabel', 'county', 'countyLabel', 'state', 'stateLabel', 'country', 'countryLabel'])

In [7]:
get_name_of_location_with_long_lat(coordinates[0])

('Q159288',
 'Santa Barbara',
 'Q108106',
 'Santa Barbara County',
 'Q99',
 'California',
 'Q30',
 'United States of America')

In [10]:
longitude_and_latitude = coordinates[0]
query =f""" 
        SELECT DISTINCT ?distance ?city ?cityLabel WHERE {{

        # Use the around service
        SERVICE wikibase:around {{ 
            # Looking for items with coordinate locations(P625)
            ?city wdt:P625 ?location . 

            # That are in a circle with a centre of with a point
            bd:serviceParam wikibase:center "Point({longitude_and_latitude})"^^geo:wktLiteral   . 
            # Where the circle has a radius of 20km
            bd:serviceParam wikibase:radius "20" . 
            bd:serviceParam wikibase:distance ?distance .
        }} .

        {{?city wdt:P31/wdt:P279* wd:Q1093829.}} # Q1093829 = city in the United States
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q4946305.}} # Q4946305 = borough in the United States
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q515.}} # Q515 = city
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q15127012.}} # Q15127012 = town in the United States
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q498162.}} # Q498162 = census-designated place in the United States
        # Use the label service to get the English label
        SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en" . 
        }}
        }}
        ORDER BY ?distance
        LIMIT 1"""
a=wikidata_query(query)
a['city.value'][0][31:], a['cityLabel.value'][0]

('Q159288', 'Santa Barbara')

In [9]:
query2 = f"""SELECT ?city ?cityLabel ?population WHERE {{
    ?city p:P1082 ?statement .
        ?statement ps:P1082 ?population .
        ?statement pq:P585 ?date .
        FILTER NOT EXISTS {{
            ?city p:P1082/pq:P585 ?date2 .
            FILTER(?date2 > ?date)
        }}# Use the label service to get the English label
        SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en" . 
        }}
        }}
        ORDER BY ?distance
        LIMIT 1"""
wikidata_query(query2)

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'SPARQL-QUERY: queryStr=SELECT ?city ?cityLabel ?population WHERE {\n    ?city p:P1082 ?statement .\n        ?statement ps:P1082 ?population .\n        ?statement pq:P585 ?date .\n        FILTER NOT EXISTS {\n            ?city p:P1082/pq:P585 ?date2 .\n            FILTER(?date2 > ?date)\n        }# Use the label service to get the English label\n        SERVICE wikibase:label {\n        bd:serviceParam wikibase:language "en" . \n        }\n        }\n        ORDER BY ?distance\n        LIMIT 1\njava.util.concurrent.TimeoutException\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:205)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:678)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat com.bigdata.rdf.sail.webapp.RESTServle

In [ ]:
def get_name_of_location_with_long_lat(longitude_and_latitude: str) -> tuple[str, str]:
    try:
        query =f""" 
        SELECT DISTINCT ?distance ?city ?cityLabel ?population ?county ?countyLabel ?state ?stateLabel ?country ?countryLabel WHERE {{

        # Use the around service
        SERVICE wikibase:around {{ 
            # Looking for items with coordinate locations(P625)
            ?city wdt:P625 ?location . 

            # That are in a circle with a centre of with a point
            bd:serviceParam wikibase:center "Point({longitude_and_latitude})"^^geo:wktLiteral   . 
            # Where the circle has a radius of 20km
            bd:serviceParam wikibase:radius "20" . 
            bd:serviceParam wikibase:distance ?distance .
        }} .

        {{?city wdt:P31/wdt:P279* wd:Q1093829.}} # Q1093829 = city in the United States
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q4946305.}} # Q4946305 = borough in the United States
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q515.}} # Q515 = city
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q15127012.}} # Q15127012 = town in the United States
        UNION
        {{?city wdt:P31/wdt:P279* wd:Q498162.}} # Q498162 = census-designated place in the United States
        
        OPTIONAL {{ ?city wdt:P131 ?county . }}
        OPTIONAL {{ ?county wdt:P131 ?state . }}
        OPTIONAL {{ ?state wdt:P131 ?country . }}
        ?city p:P1082 ?statement .
        ?statement ps:P1082 ?population .
        ?statement pq:P585 ?date .
        FILTER NOT EXISTS {{
            ?city p:P1082/pq:P585 ?date2 .
            FILTER(?date2 > ?date)
        }}
        # Use the label service to get the English label
        SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en" . 
        }}
        }}
        ORDER BY ?distance
        LIMIT 1"""
        
        a = wikidata_query(query)
        return a['city.value'][0][31:], a['cityLabel.value'][0], a['population.value'][0], a['county.value'][0][31:], a['countyLabel.value'][0], a['state.value'][0][31:], a['stateLabel.value'][0], a['country.value'][0][31:], a['countryLabel.value'][0]
    except:
        return (None, None, None, None, None, None, None, None, None)

In [4]:
location_mappings_df = pd.DataFrame(columns=['coordinates','city', 'cityLabel', 'county', 'countyLabel', 'state', 'stateLabel', 'country', 'countryLabel'])
for i in coordinates_df.itertuples():
    location_mappings_df = pd.concat([location_mappings_df,pd.DataFrame([i.coordinates] + list(get_name_of_location_with_long_lat(i.coordinates)), index=location_mappings_df.columns).T], ignore_index=True)
    

In [5]:
location_mappings_df

,coordinates,city,cityLabel,county,countyLabel,state,stateLabel,country,countryLabel
0,"-119.71,34.43",Q159288,Santa Barbara,Q108106,Santa Barbara County,Q99,California,Q30,United States of America
1,"-90.34,38.55",Q966126,Grantwood Village,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
2,"-110.88,32.22",Q18575,Tucson,Q58688,Pima County,Q816,Arizona,Q30,United States of America
3,"-75.16,39.96",None,None,None,None,None,None,None,None
4,"-75.47,40.34",Q1183476,Green Lane,Q378527,Montgomery County,Q1400,Pennsylvania,Q30,United States of America
5,"-87.06,36.27",Q616259,Ashland City,Q495072,Cheatham County,Q1509,Tennessee,Q30,United States of America
6,"-90.34,38.63",Q967881,Richmond Heights,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
7,"-82.73,27.77",Q4876370,Bear Creek,Q494556,Pinellas County,Q812,Florida,Q30,United States of America
8,"-90.32,38.57",Q962845,Shrewsbury,Q498034,St. Louis County,Q1581,Missouri,Q30,United States of America
9,"-86.77,36.21",Q23197,Nashville,Q1177705,Davidson County,Q1509,Tennessee,Q30,United States of America


In [67]:
location_mappings = pd.read_csv(get_path("location_mappings.csv"))
location_mappings

,coordinates,city,cityLabel
0,"-119.71,34.43",Q159288,Santa Barbara
1,"-90.34,38.55",Q966126,Grantwood Village
2,"-110.88,32.22",Q18575,Tucson
3,"-75.16,39.96",Q1345,Philadelphia
4,"-75.47,40.34",Q1183476,Green Lane
...,...,...,...
11410,"-82.68,27.87",Q2151669,Pinellas Park
11411,"-119.8,39.32",Q7972623,Washoe Valley
11412,"-90.05,30.02",Q34404,New Orleans
11413,"-90.23,38.66",Q38022,St. Louis


In [3]:
len(location_mappings['city'].unique()), len(location_mappings['cityLabel'].unique())

(693, 684)

In [40]:
USA_Cities = pd.read_csv(get_path("USA_Cities.csv"))
USA_Counties = pd.read_csv(get_path("USA_Counties.csv"))
USA_States = pd.read_csv(get_path("USA_States.csv"))
Canada_Cities = pd.read_csv(get_path("Canada_Cities.csv"))
Canada_States = pd.read_csv(get_path("Canada_States.csv"))

In [41]:
USA_Cities.sort_values(by=['city_iri'])

,city_iri,city,county_iri,population
14443,http://www.wikidata.org/entity/Q100,Boston,http://www.wikidata.org/entity/Q54072,675647
5473,http://www.wikidata.org/entity/Q1000030,Orange City,http://www.wikidata.org/entity/Q486118,6267
5489,http://www.wikidata.org/entity/Q1000065,Neosho,http://www.wikidata.org/entity/Q489891,12590
5450,http://www.wikidata.org/entity/Q1000082,Hampton,http://www.wikidata.org/entity/Q111694,744
5466,http://www.wikidata.org/entity/Q1000105,Chatham,http://www.wikidata.org/entity/Q115200,1529
...,...,...,...,...
5475,http://www.wikidata.org/entity/Q998914,Alva,http://www.wikidata.org/entity/Q489468,5028
5476,http://www.wikidata.org/entity/Q999293,Hewitt,http://www.wikidata.org/entity/Q111368,16026
5464,http://www.wikidata.org/entity/Q999465,Mexico Beach,http://www.wikidata.org/entity/Q488865,916
5503,http://www.wikidata.org/entity/Q999905,Buellton,http://www.wikidata.org/entity/Q108106,5161


In [42]:
USA_Cities.sort_values(by=['city_iri'])

,city_iri,city,county_iri,population
14443,http://www.wikidata.org/entity/Q100,Boston,http://www.wikidata.org/entity/Q54072,675647
5473,http://www.wikidata.org/entity/Q1000030,Orange City,http://www.wikidata.org/entity/Q486118,6267
5489,http://www.wikidata.org/entity/Q1000065,Neosho,http://www.wikidata.org/entity/Q489891,12590
5450,http://www.wikidata.org/entity/Q1000082,Hampton,http://www.wikidata.org/entity/Q111694,744
5466,http://www.wikidata.org/entity/Q1000105,Chatham,http://www.wikidata.org/entity/Q115200,1529
...,...,...,...,...
5475,http://www.wikidata.org/entity/Q998914,Alva,http://www.wikidata.org/entity/Q489468,5028
5476,http://www.wikidata.org/entity/Q999293,Hewitt,http://www.wikidata.org/entity/Q111368,16026
5464,http://www.wikidata.org/entity/Q999465,Mexico Beach,http://www.wikidata.org/entity/Q488865,916
5503,http://www.wikidata.org/entity/Q999905,Buellton,http://www.wikidata.org/entity/Q108106,5161


In [43]:
USA_Cities['city_iri'] = USA_Cities['city_iri'].apply(lambda x: x.split('/')[-1])
USA_Cities['county_iri'] = USA_Cities['county_iri'].apply(lambda x: x.split('/')[-1] if type(x) == str else x)

In [44]:
USA_Counties['county_iri'] = USA_Counties['county_iri'].apply(lambda x: x.split('/')[-1])
USA_Counties['state_iri'] = USA_Counties['state_iri'].apply(lambda x: x.split('/')[-1])

In [45]:
USA_States['state_iri'] = USA_States['state_iri'].apply(lambda x: x.split('/')[-1])
USA_States['country_iri'] = USA_States['country_iri'].apply(lambda x: x.split('/')[-1])

In [46]:
Canada_Cities['city_iri'] = Canada_Cities['city_iri'].apply(lambda x: x.split('/')[-1])
Canada_Cities['state_iri'] = Canada_Cities['state_iri'].apply(lambda x: x.split('/')[-1])

In [47]:
Canada_States['state_iri'] = Canada_States['state_iri'].apply(lambda x: x.split('/')[-1])
Canada_States['country_iri'] = Canada_States['country_iri'].apply(lambda x: x.split('/')[-1])

In [48]:
Canada_Cities

,city_iri,city,state_iri,population
0,Q2096,Edmonton,Q1951,1010899
1,Q3476253,Sherwood Park,Q1768240,68782
2,Q939127,St. Albert,Q1951,68232
3,Q253903,Beaumont,Q1951,20888


In [49]:
Canada_States

,state_iri,state,country_iri
0,Q1952,Nova Scotia,Q16
1,Q1978,Prince Edward Island,Q16
2,Q1989,Saskatchewan,Q16
3,Q2003,Newfoundland and Labrador,Q16
4,Q1904,Ontario,Q16
5,Q1948,Manitoba,Q16
6,Q1965,New Brunswick,Q16
7,Q176,Quebec,Q16
8,Q1973,British Columbia,Q16
9,Q1951,Alberta,Q16


In [50]:
Canada = Canada_Cities.merge(Canada_States, left_on='state_iri', right_on='state_iri', how='left')
Canada

,city_iri,city,state_iri,population,state,country_iri
0,Q2096,Edmonton,Q1951,1010899,Alberta,Q16
1,Q3476253,Sherwood Park,Q1768240,68782,NaN,NaN
2,Q939127,St. Albert,Q1951,68232,Alberta,Q16
3,Q253903,Beaumont,Q1951,20888,Alberta,Q16


In [51]:
USA_Cities[USA_Cities['city_iri'] == 'Q38022']

,city_iri,city,county_iri,population
14447,Q38022,St. Louis,Q1581,301578


In [52]:
USA = USA_Cities.merge(USA_Counties, left_on='county_iri', right_on='county_iri', how='left').merge(USA_States, left_on='state_iri', right_on='state_iri', how='left')

In [53]:
USA_Cities.sort_values(by='city_iri', ascending=False)

,city_iri,city,county_iri,population
5468,Q999955,Goldendale,Q820502,3453
5503,Q999905,Buellton,Q108106,5161
5464,Q999465,Mexico Beach,Q488865,916
5476,Q999293,Hewitt,Q111368,16026
5475,Q998914,Alva,Q489468,5028
...,...,...,...,...
5466,Q1000105,Chatham,Q115200,1529
5450,Q1000082,Hampton,Q111694,744
5489,Q1000065,Neosho,Q489891,12590
5473,Q1000030,Orange City,Q486118,6267


In [54]:
USA.sort_values(by=['city_iri'])

,city_iri,city,county_iri,population,county,state_iri,state,country_iri
14452,Q100,Boston,Q54072,675647,Suffolk County,Q771,Massachusetts,Q30
5475,Q1000030,Orange City,Q486118,6267,Sioux County,Q1546,Iowa,Q30
5491,Q1000065,Neosho,Q489891,12590,Newton County,Q1581,Missouri,Q30
5452,Q1000082,Hampton,Q111694,744,Dakota County,Q1527,Minnesota,Q30
5468,Q1000105,Chatham,Q115200,1529,Columbia County,Q1384,New York,Q30
...,...,...,...,...,...,...,...,...
5477,Q998914,Alva,Q489468,5028,Woods County,Q1649,Oklahoma,Q30
5478,Q999293,Hewitt,Q111368,16026,McLennan County,Q1439,Texas,Q30
5466,Q999465,Mexico Beach,Q488865,916,Bay County,Q812,Florida,Q30
5505,Q999905,Buellton,Q108106,5161,Santa Barbara County,Q99,California,Q30


In [55]:
USA_Canada = pd.concat([USA, Canada], ignore_index=True)

In [56]:
location_mappings[location_mappings['cityLabel'] == 'St. Louis']['coordinates'].value_counts()

-90.21,38.62    1
-90.21,38.6     1
-90.18,38.6     1
-90.24,38.63    1
-90.21,38.61    1
-90.25,38.59    1
-90.22,38.6     1
-90.24,38.65    1
-90.23,38.58    1
-90.18,38.62    1
-90.24,38.62    1
-90.25,38.61    1
-90.19,38.65    1
-90.19,38.64    1
-90.25,38.62    1
-90.23,38.62    1
-90.23,38.6     1
-90.21,38.65    1
-90.21,38.64    1
-90.19,38.57    1
-90.22,38.66    1
-90.2,38.6      1
-90.23,38.65    1
-90.2,38.57     1
-90.21,38.59    1
-90.18,38.64    1
-90.22,38.65    1
-90.23,38.63    1
-90.23,38.61    1
-90.18,38.63    1
-90.23,38.64    1
-90.22,38.61    1
-90.25,38.64    1
-90.19,38.62    1
-90.23,38.59    1
-90.2,38.64     1
-90.22,38.63    1
-90.24,38.6     1
-90.24,38.61    1
-90.26,38.62    1
-90.19,38.63    1
-90.26,38.63    1
-90.24,38.64    1
-90.24,38.59    1
-90.22,38.64    1
-90.25,38.63    1
-90.2,38.61     1
-90.22,38.62    1
-90.21,38.63    1
-90.19,38.61    1
-90.2,38.62     1
-90.22,38.59    1
-90.2,38.65     1
-90.2,38.63     1
-90.25,38.6     1
-90.23,38.

In [57]:
USA_Canada

,city_iri,city,county_iri,population,county,state_iri,state,country_iri
0,Q79252,Phenix City,Q501055,38816,Lee County,Q173,Alabama,Q30
1,Q79252,Phenix City,Q503329,38816,Russell County,Q173,Alabama,Q30
2,Q79276,Lamar,Q61048,1719,Johnson County,Q1612,Arkansas,Q30
3,Q79227,Bethel Heights,Q61020,3015,Benton County,Q1612,Arkansas,Q30
4,Q79286,Monette,Q61354,1506,Craighead County,Q1612,Arkansas,Q30
...,...,...,...,...,...,...,...,...
14708,Q52474,Layton,Q26631,81773,Davis County,Q829,Utah,Q30
14709,Q2096,Edmonton,NaN,1010899,NaN,Q1951,Alberta,Q16
14710,Q3476253,Sherwood Park,NaN,68782,NaN,Q1768240,NaN,NaN
14711,Q939127,St. Albert,NaN,68232,NaN,Q1951,Alberta,Q16


In [58]:
location_mappings.rename(columns={'city':'city_iri'}, inplace=True)

In [59]:
USA_Canada.sort_values(by=['city_iri'])

,city_iri,city,county_iri,population,county,state_iri,state,country_iri
14452,Q100,Boston,Q54072,675647,Suffolk County,Q771,Massachusetts,Q30
5475,Q1000030,Orange City,Q486118,6267,Sioux County,Q1546,Iowa,Q30
5491,Q1000065,Neosho,Q489891,12590,Newton County,Q1581,Missouri,Q30
5452,Q1000082,Hampton,Q111694,744,Dakota County,Q1527,Minnesota,Q30
5468,Q1000105,Chatham,Q115200,1529,Columbia County,Q1384,New York,Q30
...,...,...,...,...,...,...,...,...
5477,Q998914,Alva,Q489468,5028,Woods County,Q1649,Oklahoma,Q30
5478,Q999293,Hewitt,Q111368,16026,McLennan County,Q1439,Texas,Q30
5466,Q999465,Mexico Beach,Q488865,916,Bay County,Q812,Florida,Q30
5505,Q999905,Buellton,Q108106,5161,Santa Barbara County,Q99,California,Q30


In [60]:
df = location_mappings.merge(USA_Canada, left_on='city_iri', right_on='city_iri', how='left')

In [66]:
df.sort_values(by=['coordinates'])

,coordinates,city_iri,cityLabel,city,county_iri,population,county,state_iri,state,country_iri
10078,"-110.63,32.06",Q5903247,Rincon Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4891,"-110.66,32.01",Q1560792,Vail,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9022,"-110.66,32.09",Q5903247,Rincon Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10793,"-110.66,32.25",Q1862070,Tanque Verde,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11378,"-110.67,32.08",Q5903247,Rincon Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4494,"-90.55,38.66",Q959443,Chesterfield,Chesterfield,Q498034,49999.0,St. Louis County,Q1581,Missouri,Q30
3596,"-90.55,38.75",Q773432,St. Charles,St. Charles,Q489873,70493.0,St. Charles County,Q1581,Missouri,Q30
5074,"-90.55,38.76",Q773432,St. Charles,St. Charles,Q489873,70493.0,St. Charles County,Q1581,Missouri,Q30
9986,"-90.55,38.79",Q773432,St. Charles,St. Charles,Q489873,70493.0,St. Charles County,Q1581,Missouri,Q30


In [63]:
print(len(df['coordinates'].unique()), 
      len(df['city_iri'].unique()), 
      len(df['cityLabel'].unique()), 
      len(df['city'].unique()), 
      len(df['county_iri'].unique()), 
      len(df['population'].unique()), 
      len(df['county'].unique()), 
      len(df['state_iri'].unique()), 
      len(df['state'].unique()), 
      len(df['country_iri'].unique()))

11415 693 684 175 44 177 35 15 15 3


In [ ]:
USA_Cities.loc[USA_Cities['city_iri'].str.contains('Q966126')]

,city_iri,city,county_iri,population


In [ ]:
USA_Canada.loc[USA_Canada['county_iri'] == 'Q1581']

,city_iri,city,county_iri,population,county,state_iri,state,country_iri
4061,Q317865,Joplin,Q1581,51762,NaN,NaN,NaN,NaN
4062,Q317865,Joplin,Q1581,51762,NaN,NaN,NaN,NaN
4063,Q317865,Joplin,Q1581,51762,NaN,NaN,NaN,NaN
4064,Q317865,Joplin,Q1581,51762,NaN,NaN,NaN,NaN
6331,Q267220,Republic,Q1581,18750,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
68450,Q107260832,"Briarwood Estates, Missouri",Q1581,347,NaN,NaN,NaN,NaN
68451,Q107260850,"Lake Arrowhead, Missouri",Q1581,363,NaN,NaN,NaN,NaN
68455,Q107313405,"Shakertowne, Missouri",Q1581,625,NaN,NaN,NaN,NaN
68456,Q107313429,"Lake Timberline, Missouri",Q1581,726,NaN,NaN,NaN,NaN
